To do

Check Time Series data

Give a 2 page report to Sahameh about visualisation and indicate the data which has any problems, which is in consistant with the data from other dates
Planting date for 2018 is given in master thesis
Planting date for 2019 is May 05
Planting date for 2020 and remaining data will be given by Sahameh in the week starrting from 16th November

A lot of variations in MAT data, i.e. days to maturation.
Some have number of days while others have dates




Must do normalisation of data before training

Next step is Global Mix modelling
Sahameh will share a paper from 2020 about using hyper spectral imaging, which will be basis of Mix Modelling

In [6]:
import pandas as pd

ml_df = pd.read_csv("ml_df.csv")
ml_df

,blue_median,green_median,red_median,red_edge_median,ndvi_median,nir_median,mtci_values,evi_values,grain_yield
0,0.034152,0.085196,0.045748,0.174878,0.883373,0.734195,4.331436,0.982070,431.698672
1,0.036060,0.084995,0.045884,0.168578,0.876906,0.692295,4.268514,0.952200,409.089032
2,0.036943,0.089985,0.045336,0.180775,0.888153,0.759447,4.272559,1.017603,372.860721
3,0.030649,0.072090,0.036053,0.151221,0.906785,0.739433,5.107437,1.018870,528.219355
4,0.037586,0.085404,0.050601,0.167592,0.869756,0.713398,4.665330,0.954982,462.505958
...,...,...,...,...,...,...,...,...,...
9265,0.029813,0.115001,0.043041,0.235598,0.896805,0.778433,3.080369,1.906517,631.004782
9266,0.028875,0.116390,0.041759,0.235039,0.903283,0.792686,2.926750,1.949479,641.486148
9267,0.028778,0.119711,0.039235,0.255132,0.916507,0.884904,2.912663,2.181455,562.676888
9268,0.026632,0.106301,0.036410,0.221450,0.926392,0.858537,3.239740,2.120530,547.551879


## Machine Learning Part

In [7]:
%%time

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

Wall time: 938 ms


In [8]:
df = ml_df

# df = ml_df_std_yield


In [9]:
%%time
#==============================================================================
# Split dataframe into data and target
#==============================================================================

X = df.iloc[:,:-1].values
y = df.iloc[:,-1:].values

Wall time: 947 µs


In [10]:
%%time
y

Wall time: 0 ns


array([[431.69867173],
       [409.08903226],
       [372.86072106],
       ...,
       [562.67688805],
       [547.55187856],
       [617.82800759]])

In [15]:
%%time
#==============================================================================
# Create separate train/test splits from Main data
#==============================================================================
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.25,
                                                    random_state=14)

Wall time: 976 µs


In [16]:
%%time
# from sklearn.preprocessing import StandardScaler
# #==============================================================================
# # Scale features using StandardScaler class in scikit-learn
# #==============================================================================

# # Initialise standard scaler and compute mean and STD from training data
# sc = StandardScaler()
# sc.fit(X_train)

# # Transform (standardise) both X_train and X_test with mean and STD from
# # training data
# X_train_std = sc.transform(X_train)
# X_test_std = sc.transform(X_test)

Wall time: 0 ns


In [17]:
%%time
#==============================================================================
# Defining the function to vaiidate the model with the test data and
# get the results from regression evaluation metrices in sklearn
#==============================================================================
pred = []
accuracy = []


def test_data_regression(model, y_pred, y_test, target_state= 'actual', comments= 'no comments'):
    pred = []
    accuracy = []

    if target_state == 'transformed':
        y_pred = sc.inverse_transform(y_pred)
        y_test = sc.inverse_transform(y_test)
    #==============================================================================
    # Compute performance
    #==============================================================================

    from sklearn.metrics import mean_absolute_error
    mae = mean_absolute_error(y_test, y_pred)
    print(mae, ' mean_absolute_error')
    accuracy.append(mae)

    from sklearn.metrics import mean_squared_error
    mse = mean_squared_error(y_test, y_pred, squared=True)
    print(mse, ' mean_squared_error')
    accuracy.append(mse)

    from sklearn.metrics import mean_squared_error
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    print(rmse, ' root_mean_squared_error')
    accuracy.append(rmse)

    from sklearn.metrics import r2_score
    r2 = r2_score(y_test, y_pred)
    print(r2, ' r2_score')
    accuracy.append(r2)

    # Print accuracy computed from predictions on the test set
    print(accuracy)

    #==============================================================================
    # Append Results
    #==============================================================================
    results = []
    import datetime
    datetime = datetime.datetime.now()
    results.append((model, 'MAE = {}'.format(mae), 'MSE = {}'.format(mse),
                    'RMSE = {}'.format(rmse), 'R2 = {}'.format(r2),
                    'List = {}'.format(accuracy), datetime, target_state, comments))

    pd.DataFrame(np.asarray(results)).to_csv('results.csv',
                                             mode='a',
                                             header=None)
    pred.extend(y_pred)

Wall time: 0 ns


In [19]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_jobs=-1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
test_data_regression(model, y_pred, y_test)

<ipython-input-19-92530398dc52>:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, y_train)


42.64506101192572  mean_absolute_error
3466.6728329700027  mean_squared_error
58.87845814022309  root_mean_squared_error
0.7722338868697989  r2_score
[42.64506101192572, 3466.6728329700027, 58.87845814022309, 0.7722338868697989]


In [79]:
%%time
from sklearn.gaussian_process import GaussianProcessRegressor

model = GaussianProcessRegressor()
model

model.fit(X_train, y_train)
y_pred = model.predict(X_test)
test_data_regression(model, y_pred, y_test, 'actual', 'transformed target')



757.3234479605273  mean_absolute_error
19879011.9912847  mean_squared_error
4458.58856492553  root_mean_squared_error
-1285.5287015224958  r2_score
[757.3234479605273, 19879011.9912847, 4458.58856492553, -1285.5287015224958]
Wall time: 8.15 s


In [69]:
from sklearn.metrics import r2_score
r2 = r2_score(y_test, y_pred)
print(r2, ' r2_score')


-1285.7837651454133  r2_score


In [251]:
%%time
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=1000,
                              max_depth=250,
                              min_samples_split=5,
                              random_state=0,
                              n_jobs=-1)

model

RandomForestRegressor(max_depth=250, min_samples_split=5, n_estimators=1000,
                      n_jobs=-1, random_state=0)

In [ ]:
%%time
# Reshaping y from column ventor to 1D array
y = np.reshape(y, (1, -1))

In [252]:
%%time
from sklearn.preprocessing import MinMaxScaler

# create target scaler object
target_scaler = MinMaxScaler()
target_scaler.fit(y_train)

MinMaxScaler()

In [256]:
%%time
y_train

array([[0.80527278],
       [0.87110207],
       [0.48169135],
       ...,
       [0.86413493],
       [0.80114103],
       [0.70831504]])

In [255]:
%%time
y_test

array([[0.40024303],
       [0.75600504],
       [0.68143201],
       ...,
       [0.77390239],
       [0.49967252],
       [0.07300864]])

In [253]:
%%time


# transform target variables
y_train = target_scaler.transform(y_train)
y_test = target_scaler.transform(y_test)

In [257]:
%%time


model.fit(X_train, y_train)


<ipython-input-257-d768f88d541e>:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, y_train)


RandomForestRegressor(max_depth=250, min_samples_split=5, n_estimators=1000,
                      n_jobs=-1, random_state=0)

In [258]:
%%time

# invert transform on predictions
y_pred = model.predict(X_test)

In [260]:
%%time


y_pred = np.reshape(y_pred, (-1, 1))
y_pred

array([[0.38058973],
       [0.72374907],
       [0.71717564],
       ...,
       [0.75016846],
       [0.60480157],
       [0.21767988]])

In [261]:
%%time
y_pred = target_scaler.inverse_transform(y_pred)
y_pred

array([[351.61257326],
       [565.63091115],
       [561.53125689],
       ...,
       [582.10790565],
       [491.44686838],
       [250.01050887]])

In [263]:
%%time
y_test = target_scaler.inverse_transform(y_test)


In [264]:
%%time
test_data_regression(model, y_pred, y_test)

44.17863178170342  mean_absolute_error
3741.77485989703  mean_squared_error
61.17004871583012  root_mean_squared_error
0.7578400398368218  r2_score
[44.17863178170342, 3741.77485989703, 61.17004871583012, 0.7578400398368218]


In [232]:
%%time
y_test = target_scaler.inverse_transform(y_test)
y_test

array([[0.40024303],
       [0.75600504],
       [0.68143201],
       ...,
       [0.77390239],
       [0.49967252],
       [0.07300864]])

In [ ]:
%%time


In [191]:
%%time
from sklearn.compose import TransformedTargetRegressor


# define the target transform wrapper
wrapped_model = TransformedTargetRegressor(regressor=model, transformer=None)

In [192]:
# use the target transform wrapper

wrapped_model.fit(X_train_std, y_train)
y_pred = wrapped_model.predict(X_test_std)
test_data_regression(wrapped_model, y_pred, y_test)

44.18129274951714  mean_absolute_error
3742.084454236362  mean_squared_error
61.17257926748194  root_mean_squared_error
0.7578200035290036  r2_score
[44.18129274951714, 3742.084454236362, 61.17257926748194, 0.7578200035290036]


<ipython-input-14-92530398dc52>:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, y_train)


44.226941653718015  mean_absolute_error
3728.216447214942  mean_squared_error
61.059122555232825  root_mean_squared_error
0.7587175123726924  r2_score
[44.226941653718015, 3728.216447214942, 61.059122555232825, 0.7587175123726924]


In [67]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=10,
                              max_depth=1,
                              min_samples_split=10,
                              random_state=0,
                              n_jobs=-1)
model.fit(X_train_std, y_train)
y_pred = model.predict(X_test_std)
test_data_regression(model, y_pred, y_test)

74.65415189943113  mean_absolute_error
10483.149184427324  mean_squared_error
102.38725108345923  root_mean_squared_error
0.3215521820691706  r2_score
[74.65415189943113, 10483.149184427324, 102.38725108345923, 0.3215521820691706]


<ipython-input-67-c00bc0704170>:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train_std, y_train)


In [69]:
from sklearn.cross_decomposition import PLSRegression

model = PLSRegression(n_components=5)
model.fit(X_train_std, y_train)
y_pred = model.predict(X_test_std)
test_data_regression(model, y_pred, y_test)

65.50208431237094  mean_absolute_error
7938.921119041205  mean_squared_error
89.10062356146115  root_mean_squared_error
0.4862093808662402  r2_score
[65.50208431237094, 7938.921119041205, 89.10062356146115, 0.4862093808662402]


In [54]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV


# Define the pipeline
pipe_line = make_pipeline(RandomForestRegressor())

# Define ranges of parameter values:
param_range  = list(range(1,200))                   # For n_components
param_range2 = list(range(1, 20, 1))          # For max_iter
param_range3 = list(range(1, 15,1))                   # For max_depth
param_range4 = [x/10 for x in list(range(0, 10))]   # For learning_rate
param_range5  = list(range(5,20))                   # For n_components


# estimator.get_params().keys()
# pipe_line.get_params().keys()

gs = GridSearchCV(estimator=pipe_line, 
                  param_grid=[{'randomforestregressor__n_estimators': param_range2,
                              'randomforestregressor__max_depth': param_range3,
                               'randomforestregressor__min_samples_split': param_range3}], 
                  scoring='neg_root_mean_squared_error', 
                  cv=3,
                  n_jobs=-1)

gs = gs.fit(X, y)
print(gs.best_score_)
print(gs.best_params_)


# # Inspect AUC of parameter grid combinations
# for r, _ in enumerate(grid.cv_results_['mean_test_score']):
#     print("%0.3f +/- %0.2f %r"
#           % (grid.cv_results_['mean_test_score'][r], 
#              grid.cv_results_['std_test_score'][r] / 2.0, 
#              grid.cv_results_['params'][r]))

-128.07574617410896
{'randomforestregressor__max_depth': 3, 'randomforestregressor__min_samples_split': 5, 'randomforestregressor__n_estimators': 7}


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\pipeline.py:335: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


In [55]:
y.iloc[1].values
y

GrainYield    223.1
Name: 1, dtype: float64

In [ ]:
for i in range(len(y_pred)):
    print(y_pred[i], y[i])

In [28]:
import numpy as np
import h5py
import pandas as pd

# Visualization
import cv2
from mpl_toolkits.axes_grid1 import ImageGrid
import matplotlib.pyplot as plt

# Preprocessing
from sklearn.model_selection import train_test_split
from tensorflow.keras import backend as K

# Neural networks
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPooling2D, Dropout, Conv2DTranspose, concatenate
from tensorflow.keras.models import Model

# Training
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

# Define eval metric and loss function (DICE)

In [31]:
"""
Metric used in the competition
""" 
# Defining the dice_coef function
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

# Defining the dice_loss function
def dice_loss(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = y_true_f * y_pred_f
    score = (2. * K.sum(intersection) + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return 1. - score